In [1]:
from argparse import Namespace


In [2]:
configs = Namespace(
    is_training=1, 
    model_id='ETTm2_96_96', 
    model='Autoformer', 
    data='ETTm2', 
    root_path='./dataset/ETT-small/', 
    data_path='ETTm2.csv', 
    features='S', 
    target='OT', 
    freq='t', 
    checkpoints='./checkpoints/', 
    seq_len=96, 
    label_len=96, 
    pred_len=96, 
    bucket_size=4, 
    n_hashes=4, 
    enc_in=1, 
    dec_in=1, 
    c_out=1, 
    d_model=512, 
    n_heads=8, 
    e_layers=2, 
    d_layers=1, 
    d_ff=2048, 
    moving_avg=25, 
    factor=3, 
    distil=True, 
    dropout=0.05, 
    embed='timeF', 
    activation='gelu', 
    output_attention=False, 
    do_predict=False, 
    num_workers=10, 
    itr=1, 
    train_epochs=10, 
    batch_size=32, 
    patience=3, 
    learning_rate=0.0001, 
    des='Exp', 
    loss='mse', 
    lradj='type1', 
    use_amp=False, 
    use_gpu=True, 
    gpu=0, 
    use_multi_gpu=False, 
    devices='0,1,2,3'
)

In [3]:
from models import Autoformer

In [4]:
from exp.exp_main import Exp_Main

In [5]:
Exp = Exp_Main

In [8]:
af = Autoformer.Model(configs)

In [9]:
model_path = "checkpoints/ETTm2_96_96_Autoformer_ETTm2_ftS_sl96_ll96_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0/checkpoint.pth"

In [13]:
import torch
af.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [15]:
af.eval()

Model(
  (decomp): series_decomp(
    (moving_avg): moving_avg(
      (avg): AvgPool1d(kernel_size=(25,), stride=(1,), padding=(0,))
    )
  )
  (enc_embedding): DataEmbedding_wo_pos(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=5, out_features=512, bias=False)
    )
    (dropout): Dropout(p=0.05, inplace=False)
  )
  (dec_embedding): DataEmbedding_wo_pos(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=5, out_features=512, bias=False)
    )
    (dropout): Dropout(p=0.05, inplace=False)
  )
  (encoder): Encode

In [16]:
from data_provider import data_factory

In [17]:
data_set, data_loader = data_factory.data_provider(configs, 'test')

test 11425


In [18]:
data_loader

In [30]:
data_set[0][0].shape

(96, 1)

In [26]:
# for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(data_loader):
#     if i == 5:
#         break

#     print(f"batch_x={batch_x[0]}, batch_y={batch_y[0]}, batch_x_mark={batch_x_mark}, batch_y_mark={batch_y_mark}")

In [28]:
!pwd

/Users/ray/rayfile/auto_former/Autoformer
